# Building a basic LLM agent from scratch

In [10]:
# Inspired by https://www.deeplearning.ai/short-courses/ai-agents-in-langgraph/

In [11]:
import openai
import re
import os
from dotenv import load_dotenv
from openai import OpenAI

_ = load_dotenv()

## 1. Setup OpenAI client

In [12]:
client = OpenAI()

OPENAI_MODEL_NAME = "gpt-4o-mini"
OPENAI_MODEL_TEMPERATURE = 0

In [13]:
chat_completion = client.chat.completions.create(
    model=OPENAI_MODEL_NAME,
    messages=[{"role": "user", "content": "HI! How are you?"}]
)

In [14]:
chat_completion.choices[0].message.content

"Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?"

## 2. Create a basic agent

In [15]:
class Agent:
    def __init__(self, system="", llm_name="gpt-4o-mini", llm_temperature=0):
        self.model = llm_name
        self.temperature = llm_temperature
        self.system = system
        
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        completion = client.chat.completions.create(
                        model=self.model, 
                        temperature=self.temperature,
                        messages=self.messages)
        return completion.choices[0].message.content

In [16]:
# 
# Define a simple ReAct agent prompt, in which we specify the actions that the agent can take, the reasoning process, and the tools and functions available to the agent.
#
prompt_template = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:
"""

In [17]:
#
# Implementing the tools available to the agent
#


def calculate(what: str) -> float:
    return eval(what)


def average_dog_weight(name: str) -> str:
    if name in "Scottish Terrier": 
        return("Scottish Terriers average 20 lbs")
    elif name in "Border Collie":
        return("a Border Collies average weight is 37 lbs")
    elif name in "Toy Poodle":
        return("a toy poodles average weight is 7 lbs")
    else:
        return("An average dog weights 50 lbs")
    

tools_prompt = """
calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

average_dog_weight:
e.g. average_dog_weight: Collie
returns average weight of a dog when given the breed

Example session:

Question: How much does a Bulldog weight?
Thought: I should look the dogs weight using average_dog_weight
Action: average_dog_weight: Bulldog
PAUSE

You will be called again with this:

Observation: A Bulldog weights 51 lbs

You then output:

Answer: A bulldog weights 51 lbs
"""

In [18]:
#
# Compose full prompt
#

prompt = prompt_template + tools_prompt
print(prompt)


You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

average_dog_weight:
e.g. average_dog_weight: Collie
returns average weight of a dog when given the breed

Example session:

Question: How much does a Bulldog weigh?
Thought: I should look the dogs weight using average_dog_weight
Action: average_dog_weight: Bulldog
PAUSE

You will be called again with this:

Observation: A Bulldog weights 51 lbs

You then output:

Answer: A bulldog weights 51 lbs



### 2.2 Examples using the defined components

In [10]:
abot = Agent(prompt, OPENAI_MODEL_NAME, OPENAI_MODEL_TEMPERATURE)

In [18]:
question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""
abot(question)

"Thought: I need to find the average weights of both a Border Collie and a Scottish Terrier to calculate their combined weight. I'll start by looking up the weight of a Border Collie.  \nAction: average_dog_weight: Border Collie  \nPAUSE"

In [19]:
next_prompt = "Observation: {}".format(average_dog_weight("Border Collie"))
print(next_prompt)

Observation: a Border Collies average weight is 37 lbs


In [20]:
abot(next_prompt)

'Thought: Now that I have the weight of the Border Collie, I need to find the average weight of a Scottish Terrier to complete the calculation for their combined weight.  \nAction: average_dog_weight: Scottish Terrier  \nPAUSE'

In [21]:
next_prompt = "Observation: {}".format(average_dog_weight("Scottish Terrier"))
print(next_prompt)

Observation: Scottish Terriers average 20 lbs


In [22]:
abot(next_prompt)

'Thought: I now have the average weights for both dogs: the Border Collie weighs 37 lbs and the Scottish Terrier weighs 20 lbs. I can calculate their combined weight by adding these two values together.  \nAction: calculate: 37 + 20  \nPAUSE'

In [23]:
next_prompt = "Observation: {}".format(eval("37 + 20"))
print(next_prompt)

Observation: 57


In [24]:
abot(next_prompt)

'Answer: The combined weight of a Border Collie and a Scottish Terrier is 57 lbs.'

## 3. Create the agent execution loop

In [ ]:
#
# Specify the actions that the agent can take
#
known_actions = {
    "calculate": calculate,
    "average_dog_weight": average_dog_weight
}

action_re = re.compile('^Action: (\w+): (.*)$')  # create an action selector using regex

In [26]:
#
# This is the main function that will be used to query the agent
#
def query(question: str, max_turns: int = 5):
    """
    Query the agent with a question, and return the answer.

    This function interacts with an agent by sending a question and processing the agent's responses.
    It continues to interact with the agent for a maximum number of turns or until no further actions are required.

    Parameters:
    question (str): The initial question to ask the agent.
    max_turns (int): The maximum number of interaction turns with the agent. Default is 5.

    Raises:
    Exception: If an unknown action is encountered.

    Returns:
    None
    """
    bot = Agent(prompt, OPENAI_MODEL_NAME, OPENAI_MODEL_TEMPERATURE)
    next_prompt = question
    for i in range(max_turns):
        result = bot(next_prompt)
        print(result)

        actions = [
            action_re.match(a) 
            for a in result.split('\n') 
            if action_re.match(a)
        ]

        if actions:
            # There is an action to run
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception("Unknown action: {}: {}".format(action, action_input))
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = "Observation: {}".format(observation)
        else:
            return

### 3.1. Testing the agent loop

In [27]:
question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""
query(question)

Thought: I need to find the average weights of both a Border Collie and a Scottish Terrier to calculate their combined weight. I'll start by looking up the weight of a Border Collie.  
Action: average_dog_weight: Border Collie  
PAUSE
 -- running average_dog_weight Border Collie  
Observation: An average dog weights 50 lbs
Thought: I have the weight for the Border Collie, which is 50 lbs. Now, I need to find the average weight of a Scottish Terrier to complete the calculation for their combined weight.  
Action: average_dog_weight: Scottish Terrier  
PAUSE
 -- running average_dog_weight Scottish Terrier  
Observation: An average dog weights 50 lbs
Thought: I have the weights for both dogs now. The Border Collie weighs 50 lbs and the Scottish Terrier also weighs 50 lbs. I can now calculate their combined weight.  
Action: calculate: 50 + 50  
PAUSE
 -- running calculate 50 + 50  
Observation: 100
Answer: The combined weight of a Border Collie and a Scottish Terrier is 100 lbs.
